# Optimising the model with PEST

This example is intended to give hints as to how the user could calibrate the system using PEST.

Broadly speaking, this will involve setting up

* A PEST case/control file describing the PEST configuration for the problem. This file will need to include:
  * a list of parameters to calibrate including details of their ranges, etc
  * a list of 'observations' to match. The observations can just be objective function ideal values
  * reference to a template file, through which PEST can modify model parameters
  * reference to an instruction file, which teaches PEST how to extract the observations from the model outputs.
  * a command line for running the model
* A template file, which includes tags for each model parameter to be modified by PEST. PEST will fill this file in with the values for a given simulation.
* A script to run the model. This can be the template file
* An instruction file, which describes a model output file and how PEST can extract observations from the outputs

Given the model can be run from a bespoke Python script, this script can also be the template file that PEST modifies

This notebook includes examples of each of the require files. In the case of the template file and the control file, we generate some of the content based on the parameters we're calibrating.

PEST is available from [http://www.pesthomepage.org/Downloads.php](http://www.pesthomepage.org/Downloads.php)

In [1]:
from awrams.models import awral
import sys
import os
import pandas as pd
defaults = awral.get_default_mapping()
mapping = defaults.mapping

In [2]:
# Find all the calibratable parameters in the mapping...
# We'll calibrate n all of them, but you could equally create a subset...

params = [(k,v) for (k,v) in mapping.items() if 'Min' in v.properties]
params = [{'Name':k,'Min':v.properties['Min'],'Max':v.properties['Max'],'Value':v.args['value']} for k,v in params]
params = pd.DataFrame(params).set_index('Name')

# Here we loop through them and write out the relevant portion of the PEST Control File
param_config='\n'.join(['%s %f %f %f default_pg 1.0 0.0 1'%(k,v['Value'],v['Min'],v['Max']) for k,v in params.iterrows()])
print(param_config)

ssmax_scale 2.433261 0.500000 3.000000 default_pg 1.0 0.0 1
wslimu_hrusr 0.300000 0.150000 0.500000 default_pg 1.0 0.0 1
vc_hrusr 0.650000 0.050000 1.000000 default_pg 1.0 0.0 1
s_sls_hrudr 0.094597 0.030000 0.800000 default_pg 1.0 0.0 1
slope_coeff 0.951766 0.010000 1.000000 default_pg 1.0 0.0 1
k_rout_scale 0.050785 0.050000 3.000000 default_pg 1.0 0.0 1
lairef_hrusr 1.400000 1.300000 2.500000 default_pg 1.0 0.0 1
gfrac_max_hrudr 0.300000 0.250000 0.500000 default_pg 1.0 0.0 1
pref_gridscale 1.815294 0.100000 3.000000 default_pg 1.0 0.0 1
tsenc_hrusr 10.000000 10.000000 200.000000 default_pg 1.0 0.0 1
us0_hrusr 6.000000 1.000000 7.000000 default_pg 1.0 0.0 1
fsoilemax_hrusr 0.929708 0.200000 1.000000 default_pg 1.0 0.0 1
sdmax_scale 0.795122 0.500000 1.000000 default_pg 1.0 0.0 1
cgsmax_hrudr 0.032005 0.020000 0.050000 default_pg 1.0 0.0 1
tsenc_hrudr 60.000000 10.000000 200.000000 default_pg 1.0 0.0 1
fsoilemax_hrudr 0.227466 0.200000 1.000000 default_pg 1.0 0.0 1
gfrac_max_hrusr 0.

In [3]:
# Now, we loop through the parameters again and generate the template Python code that PEST will modify to set 
# parameter values...
param_mod_lines = '\n'.join(['mapping["%s"].args["value"] = @%s@'%(k,k) for k in params.index])
print(param_mod_lines)

mapping["ssmax_scale"].args["value"] = @ssmax_scale@
mapping["wslimu_hrusr"].args["value"] = @wslimu_hrusr@
mapping["vc_hrusr"].args["value"] = @vc_hrusr@
mapping["s_sls_hrudr"].args["value"] = @s_sls_hrudr@
mapping["slope_coeff"].args["value"] = @slope_coeff@
mapping["k_rout_scale"].args["value"] = @k_rout_scale@
mapping["lairef_hrusr"].args["value"] = @lairef_hrusr@
mapping["gfrac_max_hrudr"].args["value"] = @gfrac_max_hrudr@
mapping["pref_gridscale"].args["value"] = @pref_gridscale@
mapping["tsenc_hrusr"].args["value"] = @tsenc_hrusr@
mapping["us0_hrusr"].args["value"] = @us0_hrusr@
mapping["fsoilemax_hrusr"].args["value"] = @fsoilemax_hrusr@
mapping["sdmax_scale"].args["value"] = @sdmax_scale@
mapping["cgsmax_hrudr"].args["value"] = @cgsmax_hrudr@
mapping["tsenc_hrudr"].args["value"] = @tsenc_hrudr@
mapping["fsoilemax_hrudr"].args["value"] = @fsoilemax_hrudr@
mapping["gfrac_max_hrusr"].args["value"] = @gfrac_max_hrusr@
mapping["sla_hrudr"].args["value"] = @sla_hrudr@
mapping["cgsma

In [4]:
# Here is the template file
# It is a valid Python file, except for two points
# 1. The first line (ptf @) marks it as a PEST template file, with the @ indicating the marker used for parameters
# 2. Each of the calibrated parameters are referenced within @ @ - eg @vc_hrusr@

TEMPLATE='''ptf @
from awrams.utils import datetools as dt
from awrams.models import awral
import numpy as np
import pandas as pd

from awrams.utils import extents
from awrams.utils import mapping_types as mt
from awrams.utils.nodegraph import graph

from awrams.models import awral
from awrams.models.awral import ffi_wrapper as fw

from awrams.calibration.evaluators import Evaluator
from awrams.calibration.sce import SCEOptimizer
from awrams.calibration import objectives

from awrams.utils import catchments
import sys

defaults = awral.get_default_mapping()
mapping = defaults.mapping
dimensions = defaults.dimensions

''' + param_mod_lines + '''
cal_catchment= '421103'
time_period = dt.dates('2011')

### REMOVE: This cell just updates the mapping based on a different folder structure for climate data
mapping['precip_f'].args['pattern']='rr_*'
mapping['tmin_f'].args['pattern']='tmin_*'
mapping['tmax_f'].args['pattern']='tmax_*'
mapping['solar_f'].args['pattern']='solar_*'

### REMOVE: This class should be committed to the simulation package
FIRST=True
class OnDemandSimulator:
    def __init__(self,model,mapping):
        global FIRST
        self.input_runner = graph.ExecutionGraph(mapping)
        self.model_runner = model.get_runner(self.input_runner.get_dataspecs(True),force_build=FIRST)
        FIRST=False
       
    def run(self,period,extent,return_inputs=False,expand=True):
        coords = mt.gen_coordset(period,extent)
        iresults = self.input_runner.get_data_flat(coords,extent.mask)
        mresults = self.model_runner.run_from_mapping(iresults,coords.shape[0],\
                               extent.cell_count,True)
       
        if return_inputs:
            return mresults,iresults
        else:
            return mresults

# Load the observed streamflow data
qobs = pd.read_csv('/Users/joelrahman/Geospatial/AWRA/Calibration/Catchment_Qobs.csv',
                    parse_dates=[0])
qobs = qobs.set_index(qobs.columns[0])


obs = qobs['421103']


###
### REMOVE - This is hack (mentioned above) to shift the observed data to the time period covered by the climate data
###
obs = obs.shift(int(15*365.25),'d')


# Get the catchment as a spatial extent we can use as the bounds of the simulation
db = catchments.CatchmentDB()
spatial = db.get_by_id(cal_catchment)

mm_to_cumecs = 1e-3 * (run_results['qtot'].shape[1] * 5000 * 5000) /86400.0 # mm/day -> cumecs (to m/day -> m3/day -> ML/day)
catchment_qtot = run_results['qtot'].mean(axis=1) * mm_to_cumecs

# Set up the NSE objective calculator...
# (Precomputes the observed moment)
nse_evaluator = objectives.NSE(obs[time_period])

res = 1 - nse_evaluator(catchment_qtot_mod*mm_to_cumecs)

open('outputs.txt','w').write('objective %f\n'%res)
'''

In [5]:
# This is the PEST control/config file. Note that the list of parameters is inserted using the text computed above
PEST_CONFIG='''pcf
* control data
restart estimation
24 1 1 0 1  
1 1 single point    
10.0 2.0 0.3 0.03 8   
10.0 10.0 0.001   
0.1     
50 0.005 4 4 0.005 4   
1 1 1       
* singular value decomposition
1
24 5e-07
0
* parameter groups
default_pg relative 0.01 0.001 switch 1.5 parabolic   
* parameter data
''' + param_config + '''
* observation groups
default_og  
* observation data
objective -1 1.0 default_og
* model command line
python _run_awral.py
* model input/output
_run_awral.tpl _run_awral.py
_outputs.ins outputs.txt
'''

In [6]:
INSTRUCTION='''pif @
@objective@ !objective!
'''

In [7]:
# Here we write the files to disk...
# There would be one more file if we were using parallel pest - a run management file describing the available slaves
open('awral_pest.pst','w').write(PEST_CONFIG)
open('_run_awral.tpl','w').write(TEMPLATE)
open('_outputs.ins','w').write(INSTRUCTION)

30

In [8]:
os.system('sceua_p awral_pest')

32512